In [1]:
%cd /Users/matteoprata/PycharmProjects/lob-adverdarial-attacks-2022

/Users/matteoprata/PycharmProjects/lob-adverdarial-attacks-2022


In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.cluster import MeanShift, KMeans
import numpy as np
import pickle

np.random.seed(0)
plt.rcParams["figure.figsize"] = (16,9)

In [3]:
print(os.getcwd())

PATH_NAME = "data/stocks_picker/"
TOTAL_STOCKS = "chosen_stocks.txt"
STOCKS_FNAME = "_data_dwn_48_332__{}_2021-09-01_2022-09-01_1_1"

with open(PATH_NAME + TOTAL_STOCKS) as f:
	li_stocks = [l.strip() for l in f.readlines()]

/Users/matteoprata/PycharmProjects/lob-adverdarial-attacks-2022


In [11]:
RETURNS_FNAME = "price_return_df.dat"
if not os.path.exists(RETURNS_FNAME):

    out_df = None
    daily_midprice = pd.DataFrame()

    for i, i_stock in enumerate(li_stocks):
        try:
            print("Processing stock", i, i_stock)
            stock_path = PATH_NAME+STOCKS_FNAME.format(i_stock) + "/"
            stock_dataframe = pd.DataFrame()

            tra_days = os.listdir(stock_path)
            tra_days.sort()

            for f in tra_days:
                with open(stock_path + f) as file:
                    new_df = pd.read_csv(file)
                    stock_dataframe = pd.concat([stock_dataframe, new_df], axis=0)
            stock_dataframe = stock_dataframe.reset_index()

            # if stock_dataframe.isnull().values.any():
            #     print("skipping with nan", i, i_stock)
            #     continue

            daily_midprice[i_stock] = (stock_dataframe.loc[:, "ask_1"] + stock_dataframe.loc[:, "bid_1"]) / (2 * 10000)
        except:
            print("ignoring", i_stock)

    with open(RETURNS_FNAME, "wb") as f:
        pickle.dump(daily_midprice, f)
else:
    print("Loading pickle...")
    with open(RETURNS_FNAME, "rb") as f:
        daily_midprice = pickle.load(f)

In [12]:
daily_midprice

,AAPL,NVDA,META,LYFT,COIN,AFRM,AGNC,PARA,ERIC,BKR,...,PPBI,AAWW,IPGP,CASY,MANH,GLPG,NSIT,TTEC,CENT,LSXMB
0,152.81,224.985,15.2,47.995,260.015,94.33,16.355,29.650,11.92,22.675,...,39.335,71.805,172.035,206.375,165.765,58.76,99.545,104.995,46.455,51.365
1,152.725,224.955,15.2,48.16,260.225,94.385,16.355,29.540,11.925,22.71,...,40.2,73.285,170.775,205.025,165.75,58.755,100.125,106.13,45.860,48.695
2,152.695,224.805,15.2,48.02,260.26,94.385,16.325,29.560,11.925,22.71,...,40.2,73.285,171.355,205.025,163.585,58.83,103.625,106.13,46.190,50.49
3,152.78,224.78,15.19,48.06,260.26,94.395,16.315,29.565,11.925,22.7,...,39.335,73.285,171.36,205.020,163.585,58.83,103.435,106.06,46.195,53.65
4,152.86,224.825,15.19,48.075,260.26,94.51,16.315,29.560,11.925,22.715,...,39.665,73.285,171.36,205.030,163.585,58.79,103.43,106.13,46.190,53.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5386657,158.235,139.495,NaN,14.395,NaN,23.165,11.775,NaN,NaN,24.585,...,NaN,NaN,90.505,215.245,139.37,49.47,NaN,52.01,39.320,42.585
5386658,158.215,139.41,NaN,14.385,NaN,23.165,11.775,NaN,NaN,24.585,...,NaN,NaN,90.505,215.245,139.37,49.47,NaN,52.01,39.320,42.585
5386659,158.255,139.455,NaN,14.385,NaN,23.165,11.775,NaN,NaN,24.585,...,NaN,NaN,90.505,215.245,139.46,49.47,NaN,52.01,39.320,42.585
5386660,158.205,139.34,NaN,14.385,NaN,NaN,11.775,NaN,NaN,24.585,...,NaN,NaN,90.505,215.245,139.455,49.47,NaN,52.07,39.320,42.585


In [13]:
GRANULARITY = int(60*60*6.5)
daily_midprice = daily_midprice.iloc[::GRANULARITY, :]  # 2. samples days granularity
daily_midprice = daily_midprice.dropna(axis=0)          # 1. removes days that are not traded
daily_midprice = daily_midprice[daily_midprice != 0]
daily_midprice

,AAPL,NVDA,META,LYFT,COIN,AFRM,AGNC,PARA,ERIC,BKR,...,PPBI,AAWW,IPGP,CASY,MANH,GLPG,NSIT,TTEC,CENT,LSXMB
0,152.81,224.985,15.2,47.995,260.015,94.33,16.355,29.650,11.92,22.675,...,39.335,71.805,172.035,206.375,165.765,58.76,99.545,104.995,46.455,51.365
23400,153.845,225.22,15.34,48.87,272.645,97.4,16.325,28.925,12.075,22.76,...,38.48,75.51,171.36,203.850,172.395,60.715,100.145,110.47,45.580,50.08
46800,153.755,223.32,15.255,49.08,272.005,98.85,16.3,28.105,12.045,23.41,...,40.175,74.545,171.4,207.385,172.48,60.555,100.775,110.395,44.830,50.385
70200,155.01,228.12,15.415,48.6,276.74,94.175,16.215,28.005,11.945,23.08,...,40.18,73.71,173.185,208.350,173.55,60.935,102.525,108.76,44.825,2147.33505
93600,156.99,225.145,15.36,50.495,257.02,92.305,16.05,28.360,11.845,23.12,...,37.36,73.165,176.33,204.100,170.025,60.32,106.57,109.48,43.975,2147.33505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2901600,171.43,257.935,166.53,38.37,182.475,44.0,13.075,25.635,9.015,37.725,...,35.595,86.995,113.735,188.275,141.385,62.935,106.655,82.98,45.135,46.605
2925000,172.995,279.765,161.455,38.52,189.68,44.01,13.015,24.760,9.185,38.375,...,35.65,88.62,115.02,192.390,139.515,62.855,106.525,82.395,44.090,47.025
2948400,174.065,276.205,168.095,37.73,186.94,41.235,13.235,23.825,9.255,38.525,...,36.385,88.305,112.965,194.910,137.34,61.985,108.06,82.155,44.935,47.175
2971800,174.925,280.235,159.8,36.955,202.005,42.585,13.315,24.005,9.405,37.055,...,36.02,88.72,112.105,198.440,138.43,62.335,106.24,81.095,44.475,47.03


In [39]:
daily_midprice = daily_midprice[daily_midprice != 0]

past_mid = daily_midprice[:-1]
futu_mid = daily_midprice.shift(-1)[:-1]
daily_returns_df = (futu_mid - past_mid) / past_mid
daily_returns_df = daily_returns_df[-10 < daily_returns_df]
daily_returns_df = daily_returns_df[ 10 > daily_returns_df]
daily_returns_df

,AAPL,NVDA,META,LYFT,COIN,AFRM,AGNC,PARA,ERIC,BKR,...,PPBI,AAWW,IPGP,CASY,MANH,GLPG,NSIT,TTEC,CENT,LSXMB
0,0.006773,0.001045,0.009211,0.018231,0.048574,0.032545,-0.001834,-0.024452,0.013003,0.003749,...,-0.021736,0.051598,-0.003924,-0.012235,0.039996,0.033271,0.006027,0.052145,-0.018835,-0.025017
23400,-0.000585,-0.008436,-0.005541,0.004297,-0.002347,0.014887,-0.001531,-0.028349,-0.002484,0.028559,...,0.044049,-0.01278,0.000233,0.017341,0.000493,-0.002635,0.006291,-0.000679,-0.016455,0.00609
46800,0.008162,0.021494,0.010488,-0.00978,0.017408,-0.047294,-0.005215,-0.003558,-0.008302,-0.014097,...,0.000124,-0.011201,0.010414,0.004653,0.006204,0.006275,0.017365,-0.01481,-0.000112,NaN
70200,0.012773,-0.013041,-0.003568,0.038992,-0.071258,-0.019857,-0.010176,0.012676,-0.008372,0.001733,...,-0.070184,-0.007394,0.01816,-0.020398,-0.020311,-0.010093,0.039454,0.00662,-0.018963,0.0
93600,-0.009236,-0.007795,-0.013672,-0.018022,0.012781,-0.049564,-0.011526,0.039669,0.003377,-0.014922,...,0.015257,-0.006014,-0.018176,-0.061073,-0.005705,-0.009035,-0.051093,-0.039368,-0.083798,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2878200,0.014229,-0.028859,-0.014761,0.005899,-0.029285,0.023137,-0.009094,0.001367,-0.013136,0.041695,...,-0.039142,0.00294,-0.042876,-0.015169,-0.025469,0.011004,-0.023127,-0.04445,-0.014735,-0.001927
2901600,0.009129,0.084634,-0.030475,0.003909,0.039485,0.000227,-0.004589,-0.034133,0.018857,0.01723,...,0.001545,0.018679,0.011298,0.021856,-0.013226,-0.001271,-0.001219,-0.00705,-0.023153,0.009012
2925000,0.006185,-0.012725,0.041126,-0.020509,-0.014445,-0.063054,0.016904,-0.037763,0.007621,0.003909,...,0.020617,-0.003555,-0.017866,0.013098,-0.01559,-0.013841,0.01441,-0.002913,0.019165,0.00319
2948400,0.004941,0.014591,-0.049347,-0.020541,0.080587,0.032739,0.006045,0.007555,0.016207,-0.038157,...,-0.010032,0.0047,-0.007613,0.018111,0.007937,0.005647,-0.016842,-0.012902,-0.010237,-0.003074


In [ ]:
# WIN = 20
# stds_df = daily_returns_df.rolling(WIN).std()
# #avgs_df = daily_returns_df.rolling(WIN).mean()
# # ax = daily_returns_df.plot()
# # stds_df.plot(ax=ax)
# stds_df.iloc[:, 10:50].plot()

In [40]:
risk_return = pd.DataFrame()
risk_return["return"] = daily_returns_df.mean(axis=0)
risk_return["risk"] = daily_returns_df.std(axis=0)
risk_return["symbol"] = daily_returns_df.columns
# risk_return = risk_return.drop("ALHC")
risk_return

,return,risk,symbol
AAPL,0.001384,0.017774,AAPL
NVDA,0.002479,0.034990,NVDA
META,-0.002869,0.024255,META
LYFT,-0.000842,0.033886,LYFT
COIN,-0.000904,0.043683,COIN
...,...,...,...
GLPG,0.001069,0.032308,GLPG
NSIT,0.001004,0.024025,NSIT
TTEC,-0.001274,0.029632,TTEC
CENT,0.000093,0.023692,CENT


In [41]:
fig = px.scatter(risk_return, x='return', y='risk', text="symbol")
fig.show()

# NASDAQ Stocks with market cap from MEDIUM to MEGA

In [42]:
features = ['Last Sale', 'Market Cap', 'Country', 'Volume', 'Sector']
NASDAQ_STOCKS = "nasdaq_medium_cap_stocks.csv"

df_csv = pd.read_csv(PATH_NAME + NASDAQ_STOCKS)

df_csv['Last Sale'] = df_csv['Last Sale'].str.replace('$', '', regex=True)
df_csv['Last Sale'] = pd.to_numeric(df_csv['Last Sale'])

df_csv  # FULL csv with 675 stocks in NASDAQ, having from MEDIUM to MEGA market cap

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,AAL,American Airlines Group Inc. Common Stock,14.4150,0.4350,3.112%,9.368321e+09,United States,NaN,21917193,Consumer Discretionary,Air Freight/Delivery Services
1,AAON,AAON Inc. Common Stock,78.6450,-0.9050,-1.138%,4.183878e+09,United States,NaN,115800,Industrials,Industrial Machinery/Components
2,AAPL,Apple Inc. Common Stock,150.7514,0.5714,0.38%,2.613628e+12,United States,1980.0,50053441,Technology,Computer Manufacturing
3,AAWW,Atlas Air Worldwide Holdings NEW Common Stock,100.9650,0.1750,0.174%,2.863791e+09,United States,NaN,125662,Consumer Discretionary,Transportation Services
4,ABCB,Ameris Bancorp Common Stock,52.7400,-0.2100,-0.397%,3.657609e+09,United States,1994.0,108590,Finance,Major Banks
...,...,...,...,...,...,...,...,...,...,...,...
671,ZI,ZoomInfo Technologies Inc Common Stock,28.0700,0.7100,2.595%,1.133065e+10,United States,2020.0,2189602,Technology,Computer Software: Prepackaged Software
672,ZION,Zions Bancorporation N.A. Common Stock,51.5200,0.2600,0.507%,7.708307e+09,United States,NaN,720460,Finance,Major Banks
673,ZLAB,Zai Lab Limited American Depositary Shares,33.3500,-0.0400,-0.12%,2.513773e+09,China,2017.0,290646,Health Care,Biotechnology: Pharmaceutical Preparations
674,ZM,Zoom Video Communications Inc. Class A Common ...,76.0150,-1.1350,-1.471%,2.262550e+10,United States,2019.0,4678697,Technology,Computer Software: Prepackaged Software


In [43]:
# df_enc_cou = pd.get_dummies(df_csv['Country'])
df_enc_sec = pd.get_dummies(df_csv['Sector'])
df_encoded = pd.concat([df_csv['Volume']], axis=1) # df_enc_cou df_csv['Last Sale'],
df_encoded = df_encoded.set_index(df_csv["Symbol"])
df_encoded  # 675 stocks' last sale, market cap, volume, sector encoded for clustering

,Volume
Symbol,
AAL,21917193
AAON,115800
AAPL,50053441
AAWW,125662
ABCB,108590
...,...
ZI,2189602
ZION,720460
ZLAB,290646


### Clustering stocks features
Clustering Last Sale, Market Cap, Volume and Sector

In [ ]:
K = 5
clustering = KMeans(n_clusters=K).fit(df_encoded.values)
clustering_labels = clustering.labels_
df_csv["Cluster"] = clustering_labels
df_csv

In [ ]:
fig = px.scatter_3d(df_csv, x='Last Sale', y='Market Cap', z='Volume', color='Cluster', text="Symbol")
fig.write_html("stocks.html")
fig.show()

In [ ]:
#!open stocks.html

### Adding fetures for clustering

In [45]:
df_encoded_2 = df_encoded.join(risk_return, how="inner")
df_encoded_2 = df_encoded_2.drop("risk", axis=1)
df_encoded_2 = df_encoded_2.drop("symbol", axis=1)
df_encoded_2

,Volume,return
AAPL,50053441,0.001384
AAWW,125662,0.001904
ABMD,714165,-0.000383
ACIW,813742,-0.007532
ADI,2868841,0.000476
...,...,...
XENE,386938,0.006741
XP,4758661,-0.010464
ZG,360959,-0.003516
ZLAB,290646,-0.007162


In [46]:
K = 5
clustering = KMeans(n_clusters=K).fit(df_encoded_2.values)
clustering_labels = clustering.labels_
df_encoded_2_plot = df_encoded_2.copy(deep=True)
df_encoded_2_plot["Cluster"] = clustering_labels
df_encoded_2_plot["SYM"] = df_encoded_2_plot.index

fig = px.scatter(df_encoded_2_plot, x='Volume', y='return', color='Cluster', text=df_encoded_2_plot.index)
fig.write_html("stocks.html")
fig.show()

In [47]:
!open stocks.html

In [ ]:
df_encoded_2_plot

In [48]:
from sklearn.metrics import pairwise_distances_argmin_min
closest, _ = pairwise_distances_argmin_min(clustering.cluster_centers_, df_encoded_2.values)
df_encoded_2.iloc[closest, :]

,Volume,return
AMAT,3116139,0.000579
NVDA,39715635,0.002479
COIN,14154850,-0.000904
AAPL,50053441,0.001384
ARVN,611272,-0.001116


In [ ]:
df_encoded_2

In [ ]:
# df_encoded_2_plot["Symbol"] =
fig = px.scatter_3d(df_encoded_2_plot, x='Last Sale', y='Market Cap', z='return', color='Cluster', text=df_encoded_2_plot.index)
fig.write_html("stocks_cluster_ret_risk.html")
fig.show()

In [ ]:
!open stocks_cluster_ret_risk.html

### Correlation Matrix

In [ ]:
df_encoded_2_corr = df_encoded_2.copy(deep=True)
df_encoded_2_corr = df_encoded_2_corr.drop(["Utilities", "Basic Materials"], axis=1)
df_cor = df_encoded_2_corr.corr()
df_cor

In [ ]:
plt.imshow(df_cor, cmap='hot', interpolation='none')
plt.colorbar()
plt.xticks(range(len(df_cor)), df_cor.columns, rotation=45)
plt.yticks(range(len(df_cor)), df_cor.columns)
plt.show()

In [ ]:
df_encoded_2_plot["Symbol"] = risk_return["Symbol"]
fig = px.scatter(df_encoded_2_plot, x='return', y='risk', text="Symbol", color="Cluster")
fig.show()